In [42]:
import re
import csv
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException


In [43]:

# Set up the Selenium WebDriver (Make sure to have the appropriate driver, e.g., chromedriver, in the system PATH)
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
# Load the review page
url = "https://www.google.com/maps/place/The+Moon+Under+Water+-+JD+Wetherspoon/data=!4m7!3m6!1s0x487bb1c3f4cdafff:0x354cdeb8a6581cf8!8m2!3d53.4828699!4d-2.2468625!16s%2Fm%2F0bm8swc!19sChIJ_6_N9MOxe0gR-BxYprjeTDU?authuser=0&hl=en&rclk=1"
driver.get(url)


accept_button = driver.find_element(By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div/div/div[2]/div[1]/div[3]/div[1]/div[1]/form[2]/div/div/button/span')
accept_button.click()


# Wait for up to 10 seconds before throwing a timeout exception
wait = WebDriverWait(driver, 10)

element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]/div[2]/div[2]')))
element.click()


In [64]:
# Find the total number of reviews
total_number_of_reviews_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div[3]')
total_number_of_reviews = total_number_of_reviews_element.text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace(',','')) if ',' in total_number_of_reviews else int(total_number_of_reviews)

# Find scroll layout
scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')

# Scroll as many times as necessary to load all reviews
for i in range(0, (round(total_number_of_reviews/10 - 1))):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scrollable_div)
    time.sleep(2)


KeyboardInterrupt: 

####### First, ensure you've navigated to the Google Maps page for the location you want to scrape

####### Try to locate the element that displays the total number of reviews
try:
    total_number_of_reviews_element = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div/div[2]/div[3]')
    print(f"Total number of reviews element found: {total_number_of_reviews_element.text}")
except:
    print("Failed to locate the total number of reviews element.")

####### Try to locate the scrollable div
try:
    scrollable_div = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]')
    print("Successfully located the scrollable div.")
except:
    print("Failed to locate the scrollable div.")


In [65]:
# Get the current page's source
html_content = driver.page_source

# Save the content to a file
with open('jdwpage_relevant.html', 'w', encoding='utf-8') as file:
    file.write(html_content)


In [66]:

# Extracting the reviews
reviews = driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium")

all_reviews = []

for review in reviews:
    # Extracting star rating
    star_elements = review.find_elements(By.CSS_SELECTOR, ".kvMYJc img.vzX5Ic")
    rating = len(star_elements)
    
    # Extracting review date
    try:
        date = review.find_element(By.CSS_SELECTOR, ".rsqaWe").text
    except NoSuchElementException:
        date = "Unknown date"
    
    # Extracting review text
    try:
        review_text_elem = review.find_element(By.CSS_SELECTOR, ".wiI7pd")
        review_text = review_text_elem.text
    except NoSuchElementException:
        review_text = "No review text"
    
    # Extracting reviewer name
    reviewer_name = review.find_element(By.CSS_SELECTOR, ".d4r55").text
    
    # Extract total reviews by reviewer
    total_reviews_elem = review.find_elements(By.CSS_SELECTOR, "div.RfnDt")
    total_review_text = total_reviews_elem[0].text.split('·')[1].strip() if total_reviews_elem and '·' in total_reviews_elem[0].text else 'No reviews count'
    
    # Extracting image URLs
    img_buttons = review.find_elements(By.CSS_SELECTOR, ".Tya61d")
    img_urls = []
    for btn in img_buttons:
        style = btn.get_attribute("style")
        match = re.search(r'background-image: url\("(.*?)"\)', style)
        if match:
            img_urls.append(match.group(1))
    
    # Extracting number of images
    num_images = len(img_buttons)

    review_details = {
        "Reviewer Name": reviewer_name,
        "Review Date": date,
        "Star Rating": rating,
        "Review Text": review_text,
        "Total Reviews by Reviewer": total_review_text,
        "Number of Images": num_images,
        "Image URLs": ','.join(img_urls)
    }

    all_reviews.append(review_details)


In [67]:

# Function to save reviews to a CSV file
def save_reviews_to_csv(all_reviews, filename="jdwreviews_relevant.csv"):
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["Reviewer Name", "Review Date", "Star Rating", "Review Text", "Total Reviews by Reviewer", "Number of Images", "Image URLs"])
        writer.writeheader()
        for review in all_reviews:
            writer.writerow(review)

# Save the reviews to a CSV
save_reviews_to_csv(all_reviews)


In [6]:

###driver.close()
